In [109]:
import dash
from dash import dcc, html
from dash.dependencies import Input, Output
import plotly.express as px
import pandas as pd

data = pd.read_csv('covid_19_clean_complete.csv')

data['Date'] = pd.to_datetime(data['Date'])
data['Active'] = data['Confirmed'] - data['Deaths'] - data['Recovered']

app = dash.Dash(__name__)
app.title = "COVID-19 Dashboard"

app.layout = html.Div([
    html.H1("COVID-19 Dashboard", style={"textAlign": "center", "color": "#B0C4DE", "backgroundColor": "#002B5C", "padding": "20px"}),

    html.Div([
        html.Label("Select Country:", style={"fontWeight": "bold", "color": "#B0C4DE"}),
        dcc.Dropdown(
            id="country-dropdown",
            options=[
                {"label": country, "value": country} for country in data['Country/Region'].unique()
            ],
            value="Egypt",  # Default value
            clearable=False,
            style={"backgroundColor": "#ffffff", "color": "#B0C4DE"}
        )
    ], style={"width": "40%", "margin": "auto", "padding": "10px", "backgroundColor": "#003366", "borderRadius": "8px"}),

    dcc.Graph(id="time-series-graph", style={"backgroundColor": "#002B5C"}),

    dcc.Graph(id="map-graph", style={"backgroundColor": "#002B5C"}),

    dcc.Graph(id="scatter-graph", style={"backgroundColor": "#002B5C"}),

    dcc.Graph(id="percentage-pie-graph", style={"backgroundColor": "#002B5C"}),
], style={"fontFamily": "Arial, sans-serif", "backgroundColor": "#001F3F", "padding": "20px"})

@app.callback(
    Output("time-series-graph", "figure"),
    [Input("country-dropdown", "value")]
)
def update_time_series(selected_country):
    filtered_data = data[data['Country/Region'] == selected_country]
    fig = px.line(
        filtered_data,
        x="Date",
        y=["Confirmed", "Deaths", "Recovered"],
        labels={"value": "Cases", "variable": "Category"},
        title=f"COVID-19 Trends in {selected_country}",
        color_discrete_map={"Confirmed": "#1f77b4", "Deaths": "#ff7f0e", "Recovered": "#2ca02c"}
    )
    fig.update_layout(
        plot_bgcolor="#FAFAD2",
        paper_bgcolor="#002B5C",
        title_font_color="#B0C4DE",
        xaxis=dict(title_font_color="white", tickfont_color="white"),
        yaxis=dict(title_font_color="white", tickfont_color="white"),
        legend=dict(font=dict(color="white")),
        legend_title=dict(font=dict(color="white"))
    )
    return fig

@app.callback(
    Output("map-graph", "figure"),
    [Input("country-dropdown", "value")]
)
def update_map(selected_country):
    latest_date = data['Date'].max()
    filtered_data = data[(data['Date'] == latest_date) & (data['Country/Region'] == selected_country)]
    fig = px.scatter_geo(
        filtered_data,
        lat="Lat",
        lon="Long",
        size="Confirmed",
        color="WHO Region",
        hover_name="Province/State",
        title=f"COVID-19 Spread in {selected_country} on {latest_date.date()}",
        projection="natural earth"
    )
    fig.update_layout(
        plot_bgcolor="#FAFAD2",
        paper_bgcolor="#002B5C",
        title_font_color="#B0C4DE",
        geo=dict(bgcolor="#002B5C"),
        legend=dict(font=dict(color="white")),
        legend_title=dict(font=dict(color="white"))
    )
    return fig

@app.callback(
    Output("scatter-graph", "figure"),
    [Input("country-dropdown", "value")]
)
def update_scatter(selected_country):
    filtered_data = data[data['Country/Region'] == selected_country]
    fig = px.scatter(
        filtered_data,
        x="Date",
        y="Active",
        color="WHO Region",
        size="Confirmed",
        title=f"Active Cases Over Time in {selected_country}",
        labels={"Active": "Active Cases"}
    )
    fig.update_layout(
        plot_bgcolor="#FAFAD2",
        paper_bgcolor="#002B5C",
        title_font_color="#B0C4DE",
        xaxis=dict(title_font_color="white", tickfont_color="white"),
        yaxis=dict(title_font_color="white", tickfont_color="white"),
        legend=dict(font=dict(color="white")),
        legend_title=dict(font=dict(color="white"))
    )
    return fig

@app.callback(
    Output("percentage-pie-graph", "figure"),
    [Input("country-dropdown", "value")]
)
def update_percentage_pie(selected_country):
    latest_date = data['Date'].max()
    filtered_data = data[(data['Date'] == latest_date) & (data['Country/Region'] == selected_country)]
    total_cases = filtered_data["Confirmed"].sum()
    deaths = filtered_data["Deaths"].sum()
    active = filtered_data["Active"].sum()
    recovered = filtered_data["Recovered"].sum()

    percentages = pd.DataFrame({
        "Category": ["Deaths", "Active", "Recovered"],
        "Percentage": [deaths / total_cases * 100, active / total_cases * 100, recovered / total_cases * 100]
    })

    fig = px.pie(
        percentages,
        names="Category",
        values="Percentage",
        title=f"Percentage Distribution of Cases in {selected_country} on {latest_date.date()}"
    )
    fig.update_layout(
        plot_bgcolor="#FAFAD2",
        paper_bgcolor="#002B5C",
        title_font_color="#B0C4DE",
        legend=dict(font=dict(color="white")),
        legend_title=dict(font=dict(color="white"))
    )
    return fig

if __name__ == "__main__":
    app.run_server(debug=True)

<IPython.core.display.Javascript object>